##Домашняя работа № 2 Коллаборативная фильтрация

##Датасет MovieLens 1M

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [3]:
# загрузим данные
!wget 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

--2025-01-29 10:59:55--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  2.31MB/s    in 0.4s    

2025-01-29 10:59:55 (2.31 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [4]:
# распакуем
import zipfile

with zipfile.ZipFile('/content/ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('unpacked_files')

In [5]:
# прочитаем
links = pd.read_csv('/content/unpacked_files/ml-latest-small/links.csv')
movies = pd.read_csv('/content/unpacked_files/ml-latest-small/movies.csv')
ratings = pd.read_csv('/content/unpacked_files/ml-latest-small/ratings.csv')
tags = pd.read_csv('/content/unpacked_files/ml-latest-small/tags.csv')

In [6]:
# посмотрим
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


##Surprise

https://surpriselib.com/

Алгоритмы https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [46]:
# установим
!pip install surprise

In [99]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

Соберем и исследуем наш датасет

In [48]:
# соберем наш датасет
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [49]:
# проверим на пропуски
movies_with_ratings.isna().sum()

,0
movieId,0
title,0
genres,0
userId,0
rating,0
timestamp,0


In [50]:
# проверим на нулевые значения
movies_with_ratings.isnull().sum()

,0
movieId,0
title,0
genres,0
userId,0
rating,0
timestamp,0


In [51]:
# посмотрим на наши рейтинги
movies_with_ratings['rating'].value_counts().sort_values()

,count
rating,
0.5,1370
1.5,1791
1.0,2811
2.5,5550
2.0,7551
4.5,8551
3.5,13136
5.0,13211
3.0,20047


In [52]:
# соберем наш датасет
dataset = pd.DataFrame({'uid': movies_with_ratings.userId,
                        'iid': movies_with_ratings.title,
                        'rating': movies_with_ratings.rating
})
dataset.head(1)

,uid,iid,rating
0,1,Toy Story (1995),4.0


In [53]:
# еще раз посмотрим на рейтинги
ratings.rating.min(), ratings.rating.max()

(0.5, 5.0)

In [54]:
# сформируем объект surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [55]:
data

In [56]:
# разобъем на выборки
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [57]:
# посмотрим, что это
trainset, testset, type(testset)

(<surprise.trainset.Trainset at 0x78818badc350>,
 [(217, 'Fly Away Home (1996)', 4.0),
  (184, 'Escape Plan (2013)', 4.0),
  (263, 'Secretary (2002)', 4.5),
  (414, 'Dark Blue World (Tmavomodrý svet) (2001)', 4.0),
  (380, '50 First Dates (2004)', 3.0),
  (76, 'Star Wars: Episode I - The Phantom Menace (1999)', 2.0),
  (608, 'Cruel Intentions (1999)', 3.0),
  (79, 'Godfather, The (1972)', 5.0),
  (290, 'Simple Plan, A (1998)', 4.0),
  (599, 'Safety of Objects, The (2001)', 1.5),
  (322, 'My Cousin Vinny (1992)', 3.5),
  (226, 'Sleepy Hollow (1999)', 2.5),
  (465, 'Who Framed Roger Rabbit? (1988)', 4.0),
  (305, 'War of the Worlds (2005)', 3.5),
  (390, 'Children of a Lesser God (1986)', 4.5),
  (59, 'Witness (1985)', 5.0),
  (420, 'Monty Python Live at the Hollywood Bowl (1982)', 4.0),
  (464, 'Sudden Death (1995)', 2.5),
  (167, 'Pulp Fiction (1994)', 3.0),
  (332, 'Dune (1984)', 1.5),
  (401, 'Goofy Movie, A (1995)', 4.5),
  (263, 'Bend It Like Beckham (2002)', 4.0),
  (177, 'Tallade

In [58]:
# посмотрим на количество
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

##Модель KNNWithMeans

In [31]:
# построим модель
algo = KNNWithMeans(k=50,
                    sim_options={'name': 'cosine',
                                'user_based': True  # вычисление сходства между пользователями
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [32]:
# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8959


0.8959492188641812

In [33]:
# сделаем кросс-валидацию
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9016  0.8950  0.9139  0.8948  0.9001  0.9011  0.0070  
MAE (testset)     0.6903  0.6849  0.6970  0.6865  0.6877  0.6893  0.0043  
Fit time          0.17    0.20    0.30    0.20    0.20    0.22    0.05    
Test time         1.69    1.93    2.41    1.63    1.77    1.89    0.28    


{'test_rmse': array([0.90156213, 0.89495464, 0.91390576, 0.89480602, 0.90012097]),
 'test_mae': array([0.69028161, 0.68485976, 0.69699517, 0.68646358, 0.68767575]),
 'fit_time': (0.170823335647583,
  0.20039629936218262,
  0.3020963668823242,
  0.2044064998626709,
  0.19779062271118164),
 'test_time': (1.6947476863861084,
  1.932077169418335,
  2.407285690307617,
  1.6273205280303955,
  1.7670130729675293)}

Подберем более оптимальные параметры

In [34]:
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate

In [91]:
# зададим сетку параметров
param_grid = {#'bsl_options': {'method': ['als', 'sgd']},
              'bsl_options': {'method': ['als']},
                              #'reg': [1, 2],
                              'reg': [1],
              'k': [25, 30, 35],   # 40 по умолчанию
              #'k': [20, 26, 30],
              #'sim_options': {'name': ['msd', 'cosine', 'pearson', 'pearson_baseline'],
              'sim_options': {'name': ['pearson_baseline'],
                              #'min_support': [1, 3, 5],
                              'min_support': [1],
                              'user_based': [True]}
}

# Инициализация GridSearchCV
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=5)
# обучим модель на всем датасете
gs.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [92]:
# выведем лучшие параметры
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8976522320803847
{'bsl_options': {'method': 'als'}, 'reg': 1, 'k': 25, 'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': True}}


In [93]:
# построим и обучим нащу лучшую модель
best_param = gs.best_params['rmse']
algo = KNNWithMeans(params=best_param)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [94]:
# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8922


0.8922136195806034

In [90]:
# сделаем кросс-валидацию на всем датасете
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8986  0.8972  0.8976  0.8953  0.8947  0.8967  0.0015  
MAE (testset)     0.6879  0.6829  0.6856  0.6853  0.6841  0.6851  0.0017  
Fit time          0.16    0.13    0.13    0.13    0.13    0.14    0.01    
Test time         1.41    1.44    1.10    1.10    1.12    1.23    0.16    


{'test_rmse': array([0.89856966, 0.89718108, 0.89759631, 0.89526829, 0.89468683]),
 'test_mae': array([0.68787921, 0.68289187, 0.68556439, 0.68527457, 0.68408031]),
 'fit_time': (0.1617429256439209,
  0.13325190544128418,
  0.13450932502746582,
  0.12987446784973145,
  0.13135385513305664),
 'test_time': (1.4138121604919434,
  1.436715841293335,
  1.0950992107391357,
  1.103379249572754,
  1.1221716403961182)}

Проверим 'user_based': [True]

In [73]:
# зададим сетку параметров
param_grid = {#'bsl_options': {'method': ['als', 'sgd']},
              'bsl_options': {'method': ['als']},
                              #'reg': [1, 2],
                              'reg': [1],
              #'k': [30, 40, 50],   # 40 по умолчанию
              'k': [40, 41, 42, 43],
              #'sim_options': {'name': ['msd', 'cosine', 'pearson', 'pearson_baseline'],
              'sim_options': {'name': ['pearson_baseline'],
                              #'min_support': [1, 3, 5],
                              'min_support': [1],
                              'user_based': [False]}
}

# Инициализация GridSearchCV
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=5)
gs.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [74]:
# выведем лучшие параметры
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8840593325566444
{'bsl_options': {'method': 'als'}, 'reg': 1, 'k': 43, 'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}


In [75]:
# построим и обучим нащу лучшую модель на тестовой выборке
best_param = gs.best_params['rmse']
algo = KNNWithMeans(params=best_param)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [76]:
# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8922


0.8922136195806034

In [78]:
# сделаем кросс-валидацию на всем датасете
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8945  0.8938  0.9026  0.8972  0.9002  0.8977  0.0033  
MAE (testset)     0.6833  0.6818  0.6849  0.6880  0.6896  0.6855  0.0029  
Fit time          0.14    0.14    0.17    0.15    0.12    0.14    0.02    
Test time         1.14    1.11    2.33    1.11    1.10    1.36    0.48    


{'test_rmse': array([0.8945211 , 0.89384487, 0.90259048, 0.89717717, 0.90022917]),
 'test_mae': array([0.68327244, 0.6818498 , 0.68491055, 0.68797446, 0.6896312 ]),
 'fit_time': (0.13833022117614746,
  0.13779211044311523,
  0.16954517364501953,
  0.15007472038269043,
  0.12440013885498047),
 'test_time': (1.1371214389801025,
  1.1093456745147705,
  2.326305866241455,
  1.114478349685669,
  1.1042399406433105)}

##knns.KNNBasic

In [96]:
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate

In [97]:
# построим модель по умолчанию
algo = KNNWithMeans()
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8922


0.8922136195806034

##KNNWithZScore

In [101]:
# построим модель по умолчанию
algo = KNNWithZScore()
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8924


0.8924197320059594

##KNNBaseline

In [102]:
# построим модель по умолчанию
algo = KNNBaseline()
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8706


0.870583574696182

In [108]:
# построим модель по умолчанию
algo = KNNBaseline(user_based=True)
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8706


0.870583574696182

In [109]:
# построим модель по умолчанию
algo = KNNBaseline(user_based=False)
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8706


0.870583574696182

Уже интереснее! Исследуем подробнее

In [ ]:
# зададим сетку параметров
param_grid = {#'bsl_options': {'method': ['als', 'sgd']},
              'bsl_options': {'method': ['als']},
                              #'reg': [1, 2],
                              'reg': [1],
              #'k': [30, 40, 50],   # 40 по умолчанию
              'k': [45, 50, 55],
              #'sim_options': {'name': ['msd', 'cosine', 'pearson', 'pearson_baseline'],
              'sim_options': {'name': ['pearson_baseline'],
                              #'min_support': [1, 3, 5],
                              'min_support': [1],
                              #'user_based': [True, False]}
                              'user_based': [False]}
}

# Инициализация GridSearchCV
gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=5)
gs.fit(data)

# выведем лучшие параметры
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


In [115]:
# построим модель по умолчанию
algo = KNNBaseline(user_based=True)
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8706


0.870583574696182

In [116]:
# сделаем кросс-валидацию на всем датасете
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8787  0.8745  0.8690  0.8751  0.8780  0.8751  0.0034  
MAE (testset)     0.6706  0.6680  0.6653  0.6675  0.6705  0.6684  0.0020  
Fit time          0.44    0.53    0.46    0.46    0.46    0.47    0.03    
Test time         1.86    1.81    1.35    1.66    1.38    1.61    0.21    


{'test_rmse': array([0.87867   , 0.87454367, 0.86902356, 0.8751353 , 0.87795791]),
 'test_mae': array([0.67062156, 0.6680284 , 0.66527124, 0.66753958, 0.67048029]),
 'fit_time': (0.4360628128051758,
  0.5250716209411621,
  0.45635437965393066,
  0.46407103538513184,
  0.4609048366546631),
 'test_time': (1.8625147342681885,
  1.8145265579223633,
  1.3477542400360107,
  1.6561625003814697,
  1.3817346096038818)}

##tmp

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
movies_with_ratings.title.value_counts()

Forrest Gump (1994)                      329
Shawshank Redemption, The (1994)         317
Pulp Fiction (1994)                      307
Silence of the Lambs, The (1991)         279
Matrix, The (1999)                       278
                                        ... 
We're Back! A Dinosaur's Story (1993)      1
American Hardcore (2006)                   1
Shanghai Surprise (1986)                   1
Let's Get Harry (1986)                     1
Andrew Dice Clay: Dice Rules (1991)        1
Name: title, Length: 9719, dtype: int64

In [ ]:
movies_with_ratings.userId.value_counts()

414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
53       20
207      20
431      20
442      20
189      20
Name: userId, Length: 610, dtype: int64

In [ ]:
num_users = movies_with_ratings.userId.unique().shape[0]
num_users

610

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
movie_vector = {}

for movie, group in tqdm(movies_with_ratings.groupby('title')):
    movie_vector[movie] = np.zeros(num_users)

    for i in range(len(group.userId.values)):
        u = group.userId.values[i]
        r = group.rating.values[i]
        movie_vector[movie][int(u - 1)] = r

100%|██████████| 9719/9719 [00:02<00:00, 3800.48it/s]


In [ ]:
movie_vector['Toy Story (1995)']

array([4. , 0. , 0. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 2.5, 0. , 4.5, 3.5, 4. , 0. , 3.5, 0. , 0. , 0. , 0. , 0. ,
       3. , 0. , 0. , 0. , 5. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 0. ,
       5. , 0. , 0. , 5. , 3. , 4. , 5. , 0. , 0. , 0. , 3. , 0. , 0. ,
       0. , 3. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 0. ,
       4. , 0. , 2.5, 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0.5, 0. , 4. ,
       0. , 0. , 0. , 2.5, 0. , 0. , 0. , 4. , 0. , 0. , 3. , 3. , 4. ,
       0. , 3. , 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0. , 0. , 4. , 0. ,
       0. , 0. , 4. , 0. , 0. , 0. , 0. , 3. , 0. , 0. , 0. , 0. , 0. ,
       0. , 3.5, 0. , 4. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. , 3. ,
       0. , 2. , 0. , 3. , 4. , 0. , 4. , 0. , 0. , 3. , 4. , 0. , 0. ,
       3.5, 5. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 2. , 0. , 3. , 4. ,
       0. , 0. , 4.5, 4. , 4. , 0. , 0. , 0. , 0. , 5. , 3.5, 0. , 4.5,
       0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 4. , 0. , 0.

In [ ]:
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

In [ ]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm(movie_vector.keys()):
    if key == my_fav_film:
        continue

    titles.append(key)
    distances.append(cosine(movie_vector[my_fav_film], movie_vector[key]))

len(distances)

100%|██████████| 9719/9719 [00:02<00:00, 4553.26it/s]


9718

In [ ]:
distances[:10]

[0.9223029947150825,
 0.9300726952435743,
 0.950553928616633,
 1.0,
 1.0,
 0.968921197886033,
 0.877349727371666,
 0.9378423957720661,
 0.7578680102442159,
 0.9065861273541315]

In [ ]:
best_indexes = np.argsort(distances)[:10]
best_indexes

array([5511, 5571,  420, 4736, 5203,  405, 6864, 5204, 4322, 4737])

In [ ]:
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m)

('Matrix, The (1999)', 0.28606257328028306)
('Memento (2000)', 0.33040704494686757)
('American History X (1998)', 0.3509458374819995)
('Kill Bill: Vol. 1 (2003)', 0.360262413456623)
('Lord of the Rings: The Fellowship of the Ring, The (2001)', 0.36425580638414523)
('American Beauty (1999)', 0.3744505150386159)
('Pulp Fiction (1994)', 0.37678014788325576)
('Lord of the Rings: The Return of the King, The (2003)', 0.37798413607448667)
('Inception (2010)', 0.3845825728533704)
('Kill Bill: Vol. 2 (2004)', 0.38565615909903594)


## Surprise

https://surpriselib.com/

- Алгоритмы https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095482 sha256=1fb9739ea54c91e04b26383cb9edf5e5c63cf96aafa635936693d90cdc77126c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [ ]:
ratings.rating.min()

0.5

In [ ]:
ratings.rating.max()

5.0

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
# data_1 = Dataset.load_from_df(movies_with_ratings, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [ ]:
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

#### user_based

In [ ]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8985


0.898482528669153

In [ ]:
test_pred[:3]

[Prediction(uid=542, iid='Kill Bill: Vol. 1 (2003)', r_ui=2.5, est=3.7592260572900007, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid=469, iid='Bridge on the River Kwai, The (1957)', r_ui=4.0, est=4.168241951992395, details={'actual_k': 34, 'was_impossible': False}),
 Prediction(uid=317, iid='Scarface (1983)', r_ui=5.0, est=4.062599058241541, details={'actual_k': 49, 'was_impossible': False})]

In [ ]:
algo.predict(uid=2, iid='Fight Club (1999)')

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.474085079084162, details={'actual_k': 50, 'was_impossible': False})

#### item_based

In [ ]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute similarities between items
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred = algo.test(testset)

In [ ]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8982


0.898181158065249

In [ ]:
new_pred = algo.predict(uid=2, iid='Fight Club (1999)')
new_pred

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.2619917274785735, details={'actual_k': 24, 'was_impossible': False})

In [ ]:
accuracy.rmse(new_pred, verbose=True)

TypeError: ignored

In [ ]:
new_pred = algo.predict(uid=20000, iid='Fight Club (1999)')
new_pred

Prediction(uid=20000, iid='Fight Club (1999)', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
new_pred = algo.predict(uid=2, iid='unkown')
new_pred

Prediction(uid=2, iid='unkown', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
trainset.global_mean

3.498958694897605

#### Рекомендации

In [ ]:
def generate_recommendation(uid, model, dataset, thresh=4.5, amount=5):
    all_titles = list(dataset['iid'].values)
    users_seen_titles = dataset[dataset['uid'] == uid]['iid']
    titles = np.array(list(set(all_titles) - set(users_seen_titles)))

    np.random.shuffle(titles)

    rec_list = []
    for title in titles:
        review_prediction = model.predict(uid=uid, iid=title)
        rating = review_prediction.est

        if rating >= thresh:
            rec_list.append((title, round(rating, 2)))

            if len(rec_list) >= amount:
                return rec_list

In [ ]:
generate_recommendation(2, algo, dataset, thresh=4.8)

[('Duel in the Sun (1946)', 5.0),
 ('Hype! (1996)', 5.0),
 ('Man with the Golden Arm, The (1955)', 5.0),
 ('Four Seasons, The (1981)', 5.0),
 ('Front of the Class (2008)', 5.0)]

Дополнительные ссылки:
1. https://sales-generator.ru/blog/rekomendatelnye-sistemy/
2. https://habr.com/ru/companies/prequel/articles/567648/
3. https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101
4. https://habr.com/ru/companies/prequel/articles/573880/